In [ ]:
import numpy as np
import asammdf
import os, random 

from asammdf import MDF, Signal, set_global_option

from math import ceil
from datetime import datetime, timedelta
from pathlib import Path
from collections import Counter

set_global_option("raise_on_multiple_occurrences", False)

In [ ]:
signal_categories = {
    "Speed": [
        "Vehicle", "Wheel", "Engine", "Transmission", "Axle", "Tire", "Road", "Gear", "Brake", "Driveline",
        "ASCD", "AdaptiveCruise", "Speedometer", "Tachometer", "Odometer", "Differential", "PropShaft", "WheelHub",
        "CVJoint", "TransferCase", "Overdrive", "CruiseControl", "SpeedSensor", "WheelSpeed", "DriveShaft",
        "GroundSpeed", "WheelAngle", "AxleLoad", "TireWear", "RoadFriction", "GearShift", "BrakeForce", "DrivelineTorque",
        "TrackSpeed", "WheelSpin", "EngineSpeed", "TransSpeed", "AxleSpeed", "TireSpeed", "RoadSpeed", "GearSpeed",
        "BrakeSpeed", "DriveSpeed", "ASCDSet", "CruiseSet", "SpeedDisplay", "TachoReading", "OdoReading", "DiffSpeed",
        "PropSpeed", "HubSpeed", "CVAngle", "TransRatio", "OverdriveMode", "CruiseSpeed", "SensorSpeed"
    ],
    "Engine": [
        "RPM", "Torque", "Power", "Load", "FuelRate", "Boost", "EGT", "OilPress", "Timing", "Throttle",
        "AirFuelRatio", "KnockSensor", "Camshaft", "Crankshaft", "Piston", "Valve", "Injector", "Turbo",
        "Supercharger", "CoolingFan", "Compression", "IgnitionCoil", "ExhaustValve",
        "EngineTemp", "FuelPressure", "AirFlow", "TurboSpeed", "OilLevel", "ValveLift", "IgnitionTiming", "ExhaustFlow",
        "EngineLoad", "FuelFlow", "BoostLevel", "EGTProbe", "OilPressure", "TimingAdvance", "ThrottlePos", "AirFuelMix",
        "KnockSignal", "CamPosition", "CrankAngle", "PistonSpeed", "ValveTiming", "InjectorPulse", "TurboPressure",
        "SuperCharge", "FanSpeed", "OilGauge", "FuelGauge", "CompRatio", "CoilVoltage", "ExValvePos", "EngHeat",
        "FuelInject", "AirIntakeFlow"
    ],
    "Temperature": [
        "Coolant", "Oil", "AirIntake", "Exhaust", "Ambient", "Fuel", "Trans", "Brake", "Battery", "Cabin",
        "CatalyticConv", "Intercooler", "Radiator", "HeaterCore", "Thermostat", "OilPan", "TurboInlet", "EGRValve",
        "TransmissionFluid", "AxleOil", "DifferentialOil", "BrakeFluid", "SteeringFluid", "BatteryPack", "CabinAir",
        "ExhaustPipe", "TurboOutlet", "HeaterAir", "CoolantFlow", "OilCooler", "BatteryCell", "CabinHumidity",
        "CoolantIn", "OilOut", "IntakeAir", "ExhaustGas", "AmbientAir", "FuelLine", "TransOil", "BrakePad",
        "BattTemp", "CabinTemp", "CatConvTemp", "IntercoolTemp", "RadTemp", "HeaterTemp", "ThermoTemp", "OilPanTemp",
        "TurboInTemp", "EGRValveTemp", "TransFluid", "AxleTemp", "DiffOilTemp", "BrakeFluidTemp", "SteerFluidTemp",
        "BattPackTemp"
    ],
    "Pressure": [
        "Oil", "Fuel", "Boost", "Brake", "Tire", "Air", "Coolant", "Manifold", "Exhaust", "Hydraulic",
        "FuelRail", "TurboPressure", "Crankcase", "TransmissionPressure", "ClutchPressure", "ABSModule", "PowerSteering",
        "SuspensionAir", "FuelTank", "OilPump", "WaterPump", "BrakeBooster", "ExhaustBack", "HydraulicLine", "TireInflation",
        "AirFilter", "FuelLinePress", "TurboInlet", "CrankcaseVent", "HydraulicPump", "BrakeLinePress", "SuspensionLoad",
        "OilFlow", "FuelSys", "BoostGauge", "BrakeSys", "TirePress", "AirSys", "CoolantPress", "ManifoldPress",
        "ExhaustPress", "HydSys", "FuelRailPress", "TurboBoost", "CrankVent", "TransPress", "ClutchPress", "ABSPress",
        "SteeringPress", "SuspAir", "TankPress", "OilPumpPress", "WaterPress", "BrakeBoost", "ExBackPress", "HydLine",
        "TireInflate"
    ],
    "Electrical": [
        "Voltage", "Current", "Battery", "Alternator", "Starter", "Ignition", "Sensor", "ECU", "Ground", "Charge",
        "Fuse", "Relay", "Wiring", "Connector", "BatteryMonitor", "Inverter", "Converter", "Regulator", "Capacitor",
        "Diode", "Resistor", "IgnitionSwitch", "StarterMotor", "AlternatorOutput", "ECUPower",
        "BatteryCharge", "InverterTemp", "ConverterOutput", "RegulatorVolt", "CapacitorCharge", "DiodeDrop", "ResistorHeat",
        "VoltSys", "CurrentFlow", "BattVolt", "AltCurrent", "StartVolt", "IgnVolt", "SensorVolt", "ECUVolt",
        "GroundVolt", "ChargeLevel", "FuseRating", "RelayVolt", "WiringResist", "ConnectVolt", "BattMon", "InvertVolt",
        "ConvertVolt", "RegVolt", "CapCharge", "DiodeVolt", "ResistVal", "IgnSwitch", "StartMotor", "AltOutput",
        "ECUPowerSupply"
    ],
    "Fuel": [
        "Level", "Pressure", "Flow", "Temp", "Injection", "Consumption", "Pump", "Rail", "Tank", "Efficiency",
        "FuelFilter", "FuelLine", "FuelCap", "VaporPressure", "FuelSensor", "InjectorNozzle", "FuelPumpMotor",
        "FuelReturn", "FuelCooler", "FuelHeater", "FuelMixture", "FuelCutoff", "FuelVapor", "FuelVent", "FuelRegulator",
        "FuelQuality", "InjectionRate", "PumpSpeed", "RailTemp", "TankVent", "FuelLeak", "RegulatorFlow",
        "FuelLevel", "FuelPress", "FuelFlowRate", "FuelTemp", "InjectRate", "FuelCons", "PumpPress", "RailPress",
        "TankLevel", "FuelEff", "FilterPress", "LinePress", "CapPress", "VaporPress", "FuelSens", "InjectNozzle",
        "PumpMotor", "FuelReturnFlow", "CoolerTemp", "HeaterTemp", "MixRatio", "CutoffSwitch", "VaporLevel"
    ],
    "Brake": [
        "Pressure", "Temp", "Wear", "Force", "Fluid", "Pedal", "ABS", "Line", "Pad", "Disc", "AutoActuator",
        "BrakeCaliper", "BrakeDrum", "ParkingBrake", "EmergencyBrake", "BrakeSensor", "BrakeLight", "BrakeMaster",
        "BrakeBooster", "BrakeLine", "BrakeFluidLevel", "BrakeWearSensor", "BrakeForceDist", "BrakeTorque", "BrakeValve",
        "BrakePadWear", "CaliperTemp", "DrumWear", "ABSActuation", "LineLeak", "ValveStatus", "FluidTemp",
        "BrakePress", "BrakeHeat", "WearLevel", "ForceLevel", "FluidLevel", "PedalPos", "ABSSys", "LinePress",
        "PadWear", "DiscTemp", "ActuatorPress", "CaliperPress", "DrumTemp", "ParkBrake", "EmergBrake", "BrakeSens",
        "LightStatus", "MasterPress", "BoosterPress", "LineFlow", "FluidVol", "WearSens", "ForceDist", "TorqueLevel",
        "ValvePress"
    ],
    "Transmission": [
        "Gear", "Temp", "Speed", "Torque", "Clutch", "Fluid", "Shift", "Ratio", "Load", "Pressure",
        "TransmissionControl", "TorqueConverter", "Gearbox", "ShiftFork", "ClutchPlate", "TransmissionSensor",
        "TransmissionPump", "GearSelector", "ShiftLever", "TorqueSensor", "FluidLevel", "GearRatio", "ClutchPack", "ShiftDrum", "TransmissionCooler",
        "ShiftSpeed", "ClutchWear", "GearTemp", "PumpPressure", "SelectorPos", "LeverForce", "CoolerFlow",
        "GearPos", "TransHeat", "TransSpeed", "TransTorque", "ClutchPress", "FluidTemp", "ShiftPos", "TransRatio",
        "TransLoad", "TransPress", "ControlUnit", "ConverterTorque", "GearBoxPos", "ForkPos", "ClutchWearLevel",
        "TransSens", "PumpFlow", "SelectPos", "LeverPos", "TorqueSense", "FluidVol", "RatioLevel", "PackWear",
        "DrumPos", "CoolerTemp"
    ],
    "Suspension": [
        "Height", "Damp", "Spring", "Load", "Angle", "Shock", "Roll", "Pitch", "Yaw", "Stiffness",
        "Strut", "ControlArm", "SwayBar", "SuspensionLink", "SpringRate", "Dampener", "AirSpring", "ShockAbsorber",
        "SuspensionSensor", "RollBar", "PitchAngle", "YawRate", "StiffnessControl", "SuspensionHeight", "LoadSensor",
        "StrutLength", "ArmAngle", "BarStress", "LinkWear", "DampRate", "AirPressure", "ShockTemp",
        "SuspHeight", "DampForce", "SpringForce", "LoadLevel", "AnglePos", "ShockForce", "RollAngle", "PitchPos",
        "YawAngle", "StiffLevel", "StrutPos", "ArmPos", "SwayForce", "LinkPos", "SpringLevel", "DampLevel",
        "AirSpringPress", "ShockLevel", "SuspSens", "RollForce", "PitchRate", "YawSpeed", "StiffAdjust",
        "HeightAdjust"
    ],
    "Emissions": [
        "CO2", "NOx", "PM", "O2", "CO", "HC", "Lambda", "EGR", "DPF", "SCR",
        "EmissionSensor", "CatalystTemp", "ExhaustGas", "ParticulateFilter", "NOxSensor", "LambdaProbe", "EGRValvePos",
        "SCRFluid", "EmissionControl", "ExhaustFlow", "CO2Level", "HCLevel", "PMFilter", "O2Sensor", "SCRRate",
        "EmissionRate", "CatalystEff", "FilterLoad", "NOxRate", "ProbeVolt", "ValveFlow", "FluidLevel",
        "CO2Emission", "NOxEmission", "PMLevel", "O2Level", "COEmission", "HCEmision", "LambdaRatio", "EGRRate",
        "DPFLevel", "SCRLevel", "EmissSens", "CatTemp", "ExGasTemp", "PartFilter", "NOxSens", "LambdaSens",
        "EGRPos", "SCRFluidLevel", "EmissCtrl", "ExFlow", "CO2Rate", "HCRate", "PMFiltration", "O2Sens",
        "SCRFlow"
    ],
    "Driver": [
        "Throttle", "Brake", "Steering", "Accel", "Turn", "Clutch", "Input",
        "LaneDeparture", "NightPedestrian", "Drowsiness", "ActivePark", "BlindSpot", "CrossTraffic", "SteeringAngle",
        "DriverMonitor", "PedalPosition", "SteeringWheel", "TurnSignal", "HeadlightControl", "WiperControl", "Horn",
        "EmergencyStop", "DriverAlert", "LaneKeep", "TrafficSign", "PedestrianDetect", "BlindSpotAssist",
        "SteerTorque", "PedalForce", "AlertLevel", "TrafficSpeed", "PedDetectRange", "BlindAssistStatus",
        "ThrottlePos", "BrakePos", "SteerAngle", "AccelRate", "TurnRate", "ClutchPos",
        "LaneDepart", "NightPed", "DrowsyLevel", "ParkAssist", "BlindSpotSens",
        "CrossTrafficSens", "SteerWheel", "PedalForceLevel", "AlertStatus", "TrafficSignDetect",
        "ThrottleLevel", "BrakeLevel", "SteerLevel", "AccelLevel", "TurnLevel", "ClutchLevel", "InputRate"
    ],
    "Environment": [
        "AmbientTemp", "Humidity", "Pressure", "Altitude", "Rain", "Wind", "Light", "Noise", "RoadTemp", "AirQuality",
        "WeatherSensor", "FogLevel", "SnowDepth", "IceDetect", "WindDirection", "SunIntensity", "UVIndex", "Pollution",
        "RoadCondition", "AmbientNoise", "RainRate", "HumidityLevel", "AltitudeChange", "AirPollution", "LightSensor",
        "FogDensity", "SnowRate", "IceThickness", "WindSpeed", "SunAngle", "UVExposure", "PollutionSource",
        "AmbientHeat", "HumidLevel", "PressLevel", "AltLevel", "RainLevel", "WindLevel", "LightLevel", "NoiseLevel",
        "RoadHeat", "AirQual", "WeatherSens", "FogRate", "SnowLevel", "IceLevel", "WindDir", "SunLevel",
        "UVLevel", "PollutionLevel", "RoadCond", "AmbientSound", "RainFlow", "HumidRate", "AltChange", "AirPollute",
        "LightSens"
    ],
    "Vision": [
        "NightVision", "ThermalVision", "InfraredCamera", "DayVision", "WideAngleVision", "RearVision", "SideVision",
        "ForwardVision", "ObjectRecognition", "LaneVision", "PedestrianVision", "TrafficVision", "NightIR", "DayIR",
        "VisionProcessor", "VisionSensor", "WideField", "RearField", "SideField", "ForwardField",
        "ObjectDist", "LaneWidth", "PedHeight", "TrafficCount", "IRTemp", "ProcLoad", "SensorRange", "FieldDepth", "RearObj", "SideObj",
        "NightVis", "ThermalVis", "InfraCam", "DayVis", "WideVis", "RearVis", "SideVis", "FwdVis", "ObjRecog",
        "LaneVis", "PedVis", "TrafficVis", "NightInfra", "DayInfra", "VisProc", "VisSens", "WideAngle", "RearAngle",
        "SideAngle", "FwdAngle", "ObjDistance", "LaneSize", "PedSize", "TrafficVolume", "IRHeat", "ProcUsage"
    ],
    "Camera": [
        "FrontObjectCCD", "RearObjectCCD", "Rear", "SideCamera", "InteriorCamera", "DashboardCamera", "RoofCamera",
        "UnderCarCamera", "360Camera", "StereoCamera", "ThermalCamera", "NightCamera", "WideCamera", "ZoomCamera",
        "ParkingCamera", "BlindSpotCamera", "CrossTrafficCamera", "LaneCamera", "TrafficCamera", "ObjectCamera",
        "FrontZoom", "RearPan", "SideTilt", "IntFocus", "DashAngle", "RoofView", "UnderScan", "StereoDepth", "ThermalRange", "NightGain",
        "FrontCam", "RearCam", "SideCam", "IntCam", "DashCam", "RoofCam", "UnderCam", "360Cam", "StereoCam",
        "ThermalCam", "NightCam", "WideCam", "ZoomCam", "ParkCam", "BlindCam", "CrossCam", "LaneCam", "TrafficCam",
        "ObjCam", "FrontFocus", "RearTilt", "SideAngleCam", "IntZoom", "DashView", "RoofScan"
    ],
    "Airbag": [
        "FrontSensors", "SideCurtain", "CollisionSRS", "SideSRS", "RearAirbag", "RoofAirbag", "KneeAirbag",
        "PedestrianAirbag", "AirbagSensor", "AirbagControl", "SideImpact", "FrontImpact", "RearImpact", "RoofImpact",
        "KneeImpact", "PedestrianImpact", "AirbagDeploy", "AirbagStatus", "AirbagPressure", "AirbagTimer",
        "SensorForce", "CurtainSpeed", "SRSPower", "ImpactAngle", "DeployTime", "StatusCode", "PressureRate", "TimerReset",
        "FrontSens", "SideAirbag", "CollSRS", "SideBag", "RearBag", "RoofBag", "KneeBag", "PedBag", "AirbagSens",
        "AirbagCtrl", "SideHit", "FrontHit", "RearHit", "RoofHit", "KneeHit", "PedHit", "AirbagAct", "BagStatus",
        "BagPress", "BagTimer", "ForceSens", "CurtainRate", "SRSEnergy", "ImpactPos", "DeployRate", "StatusLevel"
    ],
    "Radar": [
        "FrontObjectLaser", "RearObjectLaser", "SideRadar", "CornerRadar", "LongRangeRadar", "ShortRangeRadar",
        "BlindSpotRadar", "CrossTrafficRadar", "ParkingRadar", "AdaptiveRadar", "DistanceRadar", "SpeedRadar",
        "ObjectRadar", "LaneRadar", "TrafficRadar", "CollisionRadar", "ObstacleRadar", "RangeSensor", "RadarBeam", "RadarPulse",
        "LaserPower", "RadarAngle", "ShortRange", "BlindDist", "CrossAngle", "ParkDist", "AdaptSpeed", "TrafficFlow", "ObstacleSize", "PulseRate",
        "FrontLaser", "RearLaser", "SideScan", "CornerScan", "LongRadar", "ShortRadar", "BlindRadar", "CrossRadar",
        "ParkRadar", "AdaptRadar", "DistRadar", "SpeedScan", "ObjRadar", "LaneScan", "TrafficScan", "CollRadar",
        "ObstRadar", "RangeScan", "BeamAngle", "PulseFreq", "LaserEnergy", "RadarPos", "ShortDist", "BlindRange",
        "CrossDist", "ParkRange"
    ],
    "Control": [
        "CentralComputer", "ECUControl", "TCUControl", "ABSControl", "TractionControl", "StabilityControl",
        "SuspensionControl", "BrakeControl", "EngineControl", "TransmissionControl", "AirbagControl", "RadarControl",
        "CameraControl", "VisionControl", "DriverAssist", "InfotainmentControl", "ClimateControl", "LightingControl",
        "SecurityControl", "DiagnosticsControl",
        "CompStatus", "ECUError", "TCUMode", "ABSRate", "TractionLevel", "StabilityAngle", "SuspMode", "BrakeMode", "EngMode", "TransMode",
        "CompCtrl", "ECUCtrl", "TCUCtrl", "ABSCtrl", "TractionCtrl", "StabCtrl", "SuspCtrl", "BrakeCtrl", "EngCtrl",
        "TransCtrl", "AirbagCtrl", "RadarCtrl", "CamCtrl", "VisionCtrl", "AssistCtrl", "InfoCtrl", "ClimateCtrl",
        "LightCtrl", "SecureCtrl", "DiagCtrl", "CompState", "ECUStatus", "TCUStatus", "ABSLevel", "TractionRate",
        "StabAngle"
    ],
    "Chassis": [
        "Frame", "ChassisRail", "CrossMember", "Subframe", "MountingPoint", "ChassisSensor", "Alignment", "TorsionBar",
        "Stabilizer", "ChassisLoad", "FrameStrength", "RailStress", "CrossStress", "SubframeLoad", "MountStress",
        "AlignmentSensor", "TorsionSensor", "StabilizerBar", "ChassisFlex", "FrameTwist",
        "RailBend", "CrossDeflect", "SubLoadDist", "MountWear", "AlignError", "TorsionAngle", "StabForce", "FlexRate", "TwistLimit",
        "FrameLoad", "RailLoad", "CrossLoad", "SubframeStress", "MountLoad", "ChassisSens", "AlignPos", "TorsionLoad",
        "StabLoad", "ChassisWeight", "FrameForce", "RailForce", "CrossForce", "SubLoad", "MountForce", "AlignSens",
        "TorsionSens", "StabBar", "ChassisBend", "FrameAngle", "RailAngle", "CrossAngle", "SubStress", "MountAngle",
        "AlignAdjust"
    ],
    "Infotainment": [
        "Display", "Touchscreen", "Audio", "Navigation", "Bluetooth", "WiFi", "USB", "HDMI", "Speaker", "Microphone",
        "InfotainmentCPU", "GPS", "Radio", "Satellite", "MediaPlayer", "VoiceControl", "GestureControl", "ScreenBrightness",
        "AudioVolume", "NavigationMap",
        "DisplayRes", "TouchSens", "AudioFreq", "NavRoute", "BTSignal", "WiFiStrength", "USBSpeed", "HDMIPort", "SpeakerLoc", "MicGain",
        "DisplaySize", "TouchResponse", "AudioLevel", "NavPath", "BTConnect", "WiFiConnect", "USBPort", "HDMIConnect",
        "SpeakerVol", "MicLevel", "InfoCPU", "GPSLoc", "RadioFreq", "SatSignal", "MediaTrack", "VoiceCmd",
        "GestureCmd", "ScreenLight", "AudioOutput", "NavMap", "DisplayMode", "TouchMode", "AudioMode", "NavMode",
        "BTMode", "WiFiMode"
    ],
    "BatteryManagement": [
        "BatteryVoltage", "BatteryCurrent", "BatteryTemp", "ChargeLevel", "StateOfCharge", "StateOfHealth", "CellVoltage",
        "CellTemp", "PackVoltage", "PackCurrent", "CoolantFlow", "ThermalRunaway", "BatterySensor", "ChargeRate",
        "DischargeRate", "BatteryBalance", "CellMonitor", "PackMonitor", "ThermalSensor", "ChargeCycle",
        "CellBal", "PackTemp", "CoolFlowRate", "RunawayTemp", "SensorVolt", "ChargeEff", "DischargeCap", "BalStatus", "MonAccuracy", "CycleCount",
        "BattVolt", "BattCurrent", "BattHeat", "ChargeLev", "SOCLevel", "SOHLevel", "CellVolt", "CellHeat",
        "PackVolt", "PackCurrent", "CoolRate", "ThermalRisk", "BattSens", "ChargeSpeed", "DischargeSpeed", "BattBal",
        "CellMon", "PackMon", "ThermalSens", "CycleNum", "CellBalance", "PackHeat", "CoolFlow", "RunawayHeat",
        "SensorVoltage", "ChargeEfficiency"
    ]
}

switch_signals = [
    "Headlight", "Wiper", "Ignition", "TurnSignal", "ParkingBrake", "EngineStart", "AC",
    "FogLight", "HighBeam", "LowBeam", "BrakeLight", "HazardLight", "InteriorLight", "TrunkLight",
    "DomeLight", "MirrorHeater", "SeatHeater", "Defroster", "RearDefogger", "Sunroof",
    "PowerWindow", "DoorLock", "TrunkRelease", "FuelCapRelease", "HoodRelease", "HornSwitch",
    "CruiseControlSwitch", "TractionControlSwitch", "StabilityControlSwitch", "LaneKeepAssist",
    "BlindSpotMonitor", "AdaptiveCruiseSwitch", "EmergencyBrakeAssist", "HillStartAssist",
    "AutoHold", "EcoMode", "SportMode", "SnowMode", "OffRoadMode", "FourWheelDrive",
    "TowMode", "EngineFan", "RadiatorFan", "Heater", "Ventilation", "AirPurifier",
    "ChildLock", "PowerMirror", "SteeringWheelHeater", "AlarmSystem",
    "HeadlightAuto", "WiperAuto", "IgnitionAuto", "TurnSignalAuto", "BrakeAuto", "EngineStop",
    "ACAuto", "FogLightAuto", "HighBeamAuto", "LowBeamAuto", "BrakeLightAuto", "HazardAuto",
    "InteriorLightAuto", "TrunkLightAuto", "DomeLightAuto", "MirrorHeaterAuto", "SeatHeaterAuto",
    "DefrosterAuto", "RearDefoggerAuto", "SunroofAuto", "WindowAuto", "DoorLockAuto",
    "TrunkReleaseAuto", "FuelCapAuto", "HoodReleaseAuto", "HornAuto", "CruiseAuto",
    "TractionAuto", "StabilityAuto", "LaneAssistAuto", "BlindSpotAuto", "CruiseAdaptAuto",
    "EmergencyBrakeAuto", "HillAssistAuto", "AutoHoldSwitch", "EcoModeSwitch", "SportModeSwitch",
    "SnowModeSwitch", "OffRoadSwitch", "FourWheelSwitch", "TowModeSwitch", "EngineFanAuto"
]

default_properties = {
    # Speed category (56 signals) 
    "Vehicle": ("km/h", 0, 250), "Wheel": ("km/h", 0, 200), "Engine": ("rpm", 700, 7000),
    "Transmission": ("rpm", 700, 7000), "Axle": ("rpm", 0, 5000), "Tire": ("bar", 1.5, 3.5),
    "Road": ("km/h", 0, 250), "Gear": ("pos", 1, 6), "Brake": ("bar", 0, 10),
    "Driveline": ("rpm", 0, 6000), "ASCD": ("km/h", 0, 200), "AdaptiveCruise": ("km/h", 0, 200),
    "Speedometer": ("km/h", 0, 250), "Tachometer": ("rpm", 0, 8000), "Odometer": ("km", 0, 1000000),
    "Differential": ("rpm", 0, 5000), "PropShaft": ("rpm", 0, 6000), "WheelHub": ("rpm", 0, 2000),
    "CVJoint": ("deg", 0, 45), "TransferCase": ("ratio", 0, 2), "Overdrive": ("bool", 0, 1),
    "CruiseControl": ("km/h", 0, 200), "SpeedSensor": ("km/h", 0, 250), "WheelSpeed": ("km/h", 0, 200),
    "DriveShaft": ("rpm", 0, 6000), "GroundSpeed": ("km/h", 0, 250), "WheelAngle": ("deg", -45, 45),
    "AxleLoad": ("kg", 0, 2000), "TireWear": ("%", 0, 100), "RoadFriction": ("coeff", 0, 1),
    "GearShift": ("pos", 1, 6), "BrakeForce": ("N", 0, 10000), "DrivelineTorque": ("Nm", 0, 500),
    "TrackSpeed": ("km/h", 0, 250), "WheelSpin": ("rpm", 0, 500), "EngineSpeed": ("rpm", 700, 7000),
    "TransSpeed": ("rpm", 700, 7000), "AxleSpeed": ("rpm", 0, 5000), "TireSpeed": ("km/h", 0, 200),
    "RoadSpeed": ("km/h", 0, 250), "GearSpeed": ("km/h", 0, 250), "BrakeSpeed": ("km/h", 0, 200),
    "DriveSpeed": ("km/h", 0, 250), "ASCDSet": ("km/h", 0, 200), "CruiseSet": ("km/h", 0, 200),
    "SpeedDisplay": ("km/h", 0, 250), "TachoReading": ("rpm", 0, 8000), "OdoReading": ("km", 0, 1000000),
    "DiffSpeed": ("rpm", 0, 5000), "PropSpeed": ("rpm", 0, 6000), "HubSpeed": ("rpm", 0, 2000),
    "CVAngle": ("deg", 0, 45), "TransRatio": ("ratio", 0, 2), "OverdriveMode": ("bool", 0, 1),
    "CruiseSpeed": ("km/h", 0, 200), "SensorSpeed": ("km/h", 0, 250),

    # Engine category (56 signals)
    "RPM": ("rpm", 700, 7000), "Torque": ("Nm", 0, 500), "Power": ("kW", 0, 300),
    "Load": ("%", 0, 100), "FuelRate": ("L/h", 0, 50), "Boost": ("bar", 0, 2.5),
    "EGT": ("°C", 200, 900), "OilPress": ("bar", 0, 5), "Timing": ("deg", -30, 30),
    "Throttle": ("%", 0, 100), "AirFuelRatio": ("ratio", 10, 20), "KnockSensor": ("V", 0, 5),
    "Camshaft": ("deg", -90, 90), "Crankshaft": ("rpm", 0, 8000), "Piston": ("mm", 0, 100),
    "Valve": ("mm", 0, 15), "Injector": ("mg/stroke", 0, 50), "Turbo": ("bar", 0, 2.5),
    "Supercharger": ("bar", 0, 2), "CoolingFan": ("rpm", 0, 3000), "Compression": ("bar", 10, 20),
    "IgnitionCoil": ("V", 0, 40), "ExhaustValve": ("mm", 0, 15), "EngineTemp": ("°C", 50, 150),
    "FuelPressure": ("bar", 0, 10), "AirFlow": ("L/s", 0, 500), "TurboSpeed": ("rpm", 0, 200000),
    "OilLevel": ("mm", 0, 100), "ValveLift": ("mm", 0, 15), "IgnitionTiming": ("deg", -30, 30),
    "ExhaustFlow": ("L/s", 0, 500), "EngineLoad": ("%", 0, 100), "FuelFlow": ("L/h", 0, 50),
    "BoostLevel": ("bar", 0, 2.5), "EGTProbe": ("°C", 200, 900), "OilPressure": ("bar", 0, 5),
    "TimingAdvance": ("deg", -30, 30), "ThrottlePos": ("%", 0, 100), "AirFuelMix": ("ratio", 10, 20),
    "KnockSignal": ("V", 0, 5), "CamPosition": ("deg", -90, 90), "CrankAngle": ("deg", 0, 360),
    "PistonSpeed": ("m/s", 0, 20), "ValveTiming": ("deg", -30, 30), "InjectorPulse": ("ms", 0, 10),
    "TurboPressure": ("bar", 0, 2.5), "SuperCharge": ("bar", 0, 2), "FanSpeed": ("rpm", 0, 3000),
    "OilGauge": ("bar", 0, 5), "FuelGauge": ("L", 0, 80), "CompRatio": ("ratio", 8, 12),
    "CoilVoltage": ("V", 0, 40), "ExValvePos": ("mm", 0, 15), "EngHeat": ("°C", 50, 150),
    "FuelInject": ("mg/stroke", 0, 50), "AirIntakeFlow": ("L/s", 0, 500),

    # Temperature category (56 signals)
    "Coolant": ("°C", 60, 120), "Oil": ("°C", 70, 120), "AirIntake": ("°C", 10, 60),
    "Exhaust": ("°C", 300, 800), "Ambient": ("°C", -20, 50), "Fuel": ("°C", 20, 80),
    "Trans": ("°C", 60, 120), "Brake": ("°C", 0, 150), "Battery": ("°C", 0, 60),
    "Cabin": ("°C", 15, 30), "CatalyticConv": ("°C", 200, 1000), "Intercooler": ("°C", 30, 100),
    "Radiator": ("°C", 60, 120), "HeaterCore": ("°C", 40, 90), "Thermostat": ("°C", 70, 100),
    "OilPan": ("°C", 50, 150), "TurboInlet": ("°C", 30, 100), "EGRValve": ("°C", 200, 600),
    "TransmissionFluid": ("°C", 50, 120), "AxleOil": ("°C", 50, 120), "DifferentialOil": ("°C", 50, 120),
    "BrakeFluid": ("°C", 0, 150), "SteeringFluid": ("°C", 0, 100), "BatteryPack": ("°C", 0, 60),
    "CabinAir": ("°C", 15, 30), "ExhaustPipe": ("°C", 300, 800), "TurboOutlet": ("°C", 30, 100),
    "HeaterAir": ("°C", 40, 90), "CoolantFlow": ("°C", 60, 120), "OilCooler": ("°C", 50, 120),
    "BatteryCell": ("°C", 0, 60), "CabinHumidity": ("%", 0, 100), "CoolantIn": ("°C", 60, 120),
    "OilOut": ("°C", 70, 120), "IntakeAir": ("°C", 10, 60), "ExhaustGas": ("°C", 300, 800),
    "AmbientAir": ("°C", -20, 50), "FuelLine": ("°C", 20, 80), "TransOil": ("°C", 60, 120),
    "BrakePad": ("°C", 0, 150), "BattTemp": ("°C", 0, 60), "CabinTemp": ("°C", 15, 30),
    "CatConvTemp": ("°C", 200, 1000), "IntercoolTemp": ("°C", 30, 100), "RadTemp": ("°C", 60, 120),
    "HeaterTemp": ("°C", 40, 90), "ThermoTemp": ("°C", 70, 100), "OilPanTemp": ("°C", 50, 150),
    "TurboInTemp": ("°C", 30, 100), "EGRValveTemp": ("°C", 200, 600), "TransFluid": ("°C", 50, 120),
    "AxleTemp": ("°C", 50, 120), "DiffOilTemp": ("°C", 50, 120), "BrakeFluidTemp": ("°C", 0, 150),
    "SteerFluidTemp": ("°C", 0, 100), "BattPackTemp": ("°C", 0, 60),

    # Pressure category (56 signals) 
    "Oil": ("bar", 0, 5), "Fuel": ("bar", 0, 10), "Boost": ("bar", 0, 2.5),
    "Brake": ("bar", 0, 10), "Tire": ("bar", 1.5, 3.5), "Air": ("bar", 0, 10),
    "Coolant": ("bar", 0, 5), "Manifold": ("bar", 0, 2.5), "Exhaust": ("bar", 0, 2),
    "Hydraulic": ("bar", 0, 10), "FuelRail": ("bar", 0, 2000), "TurboPressure": ("bar", 0, 2.5),
    "Crankcase": ("bar", -0.5, 0.5), "TransmissionPressure": ("bar", 0, 10), "ClutchPressure": ("bar", 0, 10),
    "ABSModule": ("bar", 0, 10), "PowerSteering": ("bar", 0, 100), "SuspensionAir": ("bar", 0, 10),
    "FuelTank": ("bar", -0.1, 0.5), "OilPump": ("bar", 0, 5), "WaterPump": ("bar", 0, 5),
    "BrakeBooster": ("bar", 0, 10), "ExhaustBack": ("bar", 0, 2), "HydraulicLine": ("bar", 0, 10),
    "TireInflation": ("bar", 1.5, 3.5), "AirFilter": ("bar", 0, 2), "FuelLinePress": ("bar", 0, 10),
    "TurboInlet": ("bar", 0, 2.5), "CrankcaseVent": ("bar", -0.5, 0.5), "HydraulicPump": ("bar", 0, 10),
    "BrakeLinePress": ("bar", 0, 10), "SuspensionLoad": ("bar", 0, 10), "OilFlow": ("bar", 0, 5),
    "FuelSys": ("bar", 0, 10), "BoostGauge": ("bar", 0, 2.5), "BrakeSys": ("bar", 0, 10),
    "TirePress": ("bar", 1.5, 3.5), "AirSys": ("bar", 0, 10), "CoolantPress": ("bar", 0, 5),
    "ManifoldPress": ("bar", 0, 2.5), "ExhaustPress": ("bar", 0, 2), "HydSys": ("bar", 0, 10),
    "FuelRailPress": ("bar", 0, 2000), "TurboBoost": ("bar", 0, 2.5), "CrankVent": ("bar", -0.5, 0.5),
    "TransPress": ("bar", 0, 10), "ClutchPress": ("bar", 0, 10), "ABSPress": ("bar", 0, 10),
    "SteeringPress": ("bar", 0, 100), "SuspAir": ("bar", 0, 10), "TankPress": ("bar", -0.1, 0.5),
    "OilPumpPress": ("bar", 0, 5), "WaterPress": ("bar", 0, 5), "BrakeBoost": ("bar", 0, 10),
    "ExBackPress": ("bar", 0, 2), "HydLine": ("bar", 0, 10), "TireInflate": ("bar", 1.5, 3.5),

    # Electrical category (56 signals) 
    "Voltage": ("V", 0, 24), "Current": ("A", 0, 200), "Battery": ("V", 0, 24),
    "Alternator": ("V", 13, 15), "Starter": ("V", 0, 12), "Ignition": ("V", 0, 24),
    "Sensor": ("V", 0, 5), "ECU": ("V", 0, 24), "Ground": ("V", 0, 1),
    "Charge": ("%", 0, 100), "Fuse": ("A", 0, 50), "Relay": ("V", 0, 24),
    "Wiring": ("ohm", 0, 1), "Connector": ("V", 0, 24), "BatteryMonitor": ("V", 0, 24),
    "Inverter": ("V", 0, 400), "Converter": ("V", 0, 400), "Regulator": ("V", 12, 15),
    "Capacitor": ("F", 0, 0.001), "Diode": ("V", 0, 1), "Resistor": ("ohm", 0, 1000),
    "IgnitionSwitch": ("V", 0, 12), "StarterMotor": ("V", 0, 12), "AlternatorOutput": ("V", 13, 15),
    "ECUPower": ("V", 0, 24), "BatteryCharge": ("%", 0, 100), "InverterTemp": ("°C", 0, 100),
    "ConverterOutput": ("V", 0, 400), "RegulatorVolt": ("V", 12, 15), "CapacitorCharge": ("C", 0, 0.001),
    "DiodeDrop": ("V", 0, 1), "ResistorHeat": ("°C", 0, 100), "VoltSys": ("V", 0, 24),
    "CurrentFlow": ("A", 0, 200), "BattVolt": ("V", 0, 24), "AltCurrent": ("A", 0, 200),
    "StartVolt": ("V", 0, 12), "IgnVolt": ("V", 0, 24), "SensorVolt": ("V", 0, 5),
    "ECUVolt": ("V", 0, 24), "GroundVolt": ("V", 0, 1), "ChargeLevel": ("%", 0, 100),
    "FuseRating": ("A", 0, 50), "RelayVolt": ("V", 0, 24), "WiringResist": ("ohm", 0, 1),
    "ConnectVolt": ("V", 0, 24), "BattMon": ("V", 0, 24), "InvertVolt": ("V", 0, 400),
    "ConvertVolt": ("V", 0, 400), "RegVolt": ("V", 12, 15), "CapCharge": ("F", 0, 0.001),
    "DiodeVolt": ("V", 0, 1), "ResistVal": ("ohm", 0, 1000), "IgnSwitch": ("V", 0, 12),
    "StartMotor": ("V", 0, 12), "AltOutput": ("V", 13, 15), "ECUPowerSupply": ("V", 0, 24),

    # Fuel category (56 signals)
    "Level": ("%", 0, 100), "Pressure": ("bar", 0, 10), "Flow": ("L/h", 0, 50),
    "Temp": ("°C", 20, 80), "Injection": ("mg/stroke", 0, 50), "Consumption": ("L/100km", 0, 20),
    "Pump": ("bar", 0, 10), "Rail": ("bar", 0, 2000), "Tank": ("L", 0, 80),
    "Efficiency": ("%", 0, 100), "FuelFilter": ("bar", 0, 2), "FuelLine": ("bar", 0, 10),
    "FuelCap": ("bar", 0, 0.5), "VaporPressure": ("bar", 0, 0.5), "FuelSensor": ("V", 0, 5),
    "InjectorNozzle": ("mg/stroke", 0, 50), "FuelPumpMotor": ("V", 0, 12), "FuelReturn": ("L/h", 0, 20),
    "FuelCooler": ("°C", 20, 80), "FuelHeater": ("°C", 20, 80), "FuelMixture": ("ratio", 10, 20),
    "FuelCutoff": ("bool", 0, 1), "FuelVapor": ("g", 0, 10), "FuelVent": ("L/h", 0, 10),
    "FuelRegulator": ("bar", 0, 10), "FuelQuality": ("index", 0, 100), "InjectionRate": ("mg/s", 0, 50),
    "PumpSpeed": ("rpm", 0, 5000), "RailTemp": ("°C", 20, 80), "TankVent": ("L/h", 0, 10),
    "FuelLeak": ("L/h", 0, 1), "RegulatorFlow": ("L/h", 0, 20), "FuelLevel": ("%", 0, 100),
    "FuelPress": ("bar", 0, 10), "FuelFlowRate": ("L/h", 0, 50), "FuelTemp": ("°C", 20, 80),
    "InjectRate": ("mg/stroke", 0, 50), "FuelCons": ("L/100km", 0, 20), "PumpPress": ("bar", 0, 10),
    "RailPress": ("bar", 0, 2000), "TankLevel": ("L", 0, 80), "FuelEff": ("%", 0, 100),
    "FilterPress": ("bar", 0, 2), "LinePress": ("bar", 0, 10), "CapPress": ("bar", 0, 0.5),
    "VaporPress": ("bar", 0, 0.5), "FuelSens": ("V", 0, 5), "InjectNozzle": ("mg/stroke", 0, 50),
    "PumpMotor": ("V", 0, 12), "FuelReturnFlow": ("L/h", 0, 20), "CoolerTemp": ("°C", 20, 80),
    "HeaterTemp": ("°C", 20, 80), "MixRatio": ("ratio", 10, 20), "CutoffSwitch": ("bool", 0, 1),
    "VaporLevel": ("g", 0, 10),

    # Brake category (56 signals)
    "Pressure": ("bar", 0, 10), "Temp": ("°C", 0, 150), "Wear": ("%", 0, 100),
    "Force": ("N", 0, 10000), "Fluid": ("mm", 0, 100), "Pedal": ("%", 0, 100),
    "ABS": ("bar", 0, 10), "Line": ("bar", 0, 10), "Pad": ("mm", 0, 20),
    "Disc": ("mm", 0, 30), "AutoActuator": ("bar", 0, 10), "BrakeCaliper": ("bar", 0, 10),
    "BrakeDrum": ("mm", 0, 30), "ParkingBrake": ("bool", 0, 1), "EmergencyBrake": ("bool", 0, 1),
    "BrakeSensor": ("V", 0, 5), "BrakeLight": ("bool", 0, 1), "BrakeMaster": ("bar", 0, 10),
    "BrakeBooster": ("bar", 0, 10), "BrakeLine": ("bar", 0, 10), "BrakeFluidLevel": ("mm", 0, 100),
    "BrakeWearSensor": ("V", 0, 5), "BrakeForceDist": ("N", 0, 5000), "BrakeTorque": ("Nm", 0, 500),
    "BrakeValve": ("bar", 0, 10), "BrakePadWear": ("mm", 0, 20), "CaliperTemp": ("°C", 0, 150),
    "DrumWear": ("mm", 0, 30), "ABSActuation": ("bar", 0, 10), "LineLeak": ("L/h", 0, 1),
    "ValveStatus": ("bool", 0, 1), "FluidTemp": ("°C", 0, 150), "BrakePress": ("bar", 0, 10),
    "BrakeHeat": ("°C", 0, 150), "WearLevel": ("%", 0, 100), "ForceLevel": ("N", 0, 10000),
    "FluidLevel": ("mm", 0, 100), "PedalPos": ("%", 0, 100), "ABSSys": ("bar", 0, 10),
    "LinePress": ("bar", 0, 10), "PadWear": ("mm", 0, 20), "DiscTemp": ("°C", 0, 150),
    "ActuatorPress": ("bar", 0, 10), "CaliperPress": ("bar", 0, 10), "DrumTemp": ("°C", 0, 150),
    "ParkBrake": ("bool", 0, 1), "EmergBrake": ("bool", 0, 1), "BrakeSens": ("V", 0, 5),
    "LightStatus": ("bool", 0, 1), "MasterPress": ("bar", 0, 10), "BoosterPress": ("bar", 0, 10),
    "LineFlow": ("L/h", 0, 1), "FluidVol": ("mm", 0, 100), "WearSens": ("V", 0, 5),
    "ForceDist": ("N", 0, 5000), "TorqueLevel": ("Nm", 0, 500), "ValvePress": ("bar", 0, 10),

    # Transmission category (56 signals)
    "Gear": ("pos", 1, 6), "Temp": ("°C", 50, 120), "Speed": ("rpm", 0, 6000),
    "Torque": ("Nm", 0, 500), "Clutch": ("%", 0, 100), "Fluid": ("mm", 0, 100),
    "Shift": ("pos", 1, 6), "Ratio": ("ratio", 0, 5), "Load": ("%", 0, 100),
    "Pressure": ("bar", 0, 10), "TransmissionControl": ("bool", 0, 1), "TorqueConverter": ("Nm", 0, 500),
    "Gearbox": ("pos", 1, 6), "ShiftFork": ("mm", 0, 50), "ClutchPlate": ("mm", 0, 10),
    "TransmissionSensor": ("V", 0, 5), "TransmissionPump": ("bar", 0, 10), "GearSelector": ("pos", 1, 6),
    "ShiftLever": ("pos", 1, 6), "TorqueSensor": ("Nm", 0, 500), "FluidLevel": ("mm", 0, 100),
    "GearRatio": ("ratio", 0, 5), "ClutchPack": ("mm", 0, 10), "ShiftDrum": ("deg", 0, 360),
    "TransmissionCooler": ("°C", 50, 120), "ShiftSpeed": ("km/h", 0, 200), "ClutchWear": ("%", 0, 100),
    "GearTemp": ("°C", 50, 120), "PumpPressure": ("bar", 0, 10), "SelectorPos": ("pos", 1, 6),
    "LeverForce": ("N", 0, 100), "CoolerFlow": ("L/min", 0, 20), "GearPos": ("pos", 1, 6),
    "TransHeat": ("°C", 50, 120), "TransSpeed": ("rpm", 0, 6000), "TransTorque": ("Nm", 0, 500),
    "ClutchPress": ("%", 0, 100), "FluidTemp": ("°C", 50, 120), "ShiftPos": ("pos", 1, 6),
    "TransRatio": ("ratio", 0, 5), "TransLoad": ("%", 0, 100), "TransPress": ("bar", 0, 10),
    "ControlUnit": ("bool", 0, 1), "ConverterTorque": ("Nm", 0, 500), "GearBoxPos": ("pos", 1, 6),
    "ForkPos": ("mm", 0, 50), "ClutchWearLevel": ("mm", 0, 10), "TransSens": ("V", 0, 5),
    "PumpFlow": ("L/min", 0, 20), "SelectPos": ("pos", 1, 6), "LeverPos": ("pos", 1, 6),
    "TorqueSense": ("Nm", 0, 500), "FluidVol": ("mm", 0, 100), "RatioLevel": ("ratio", 0, 5),
    "PackWear": ("mm", 0, 10), "DrumPos": ("deg", 0, 360), "CoolerTemp": ("°C", 50, 120),

    # Suspension category (56 signals)
    "Height": ("mm", 0, 200), "Damp": ("N", 0, 5000), "Spring": ("N/mm", 0, 100),
    "Load": ("kg", 0, 2000), "Angle": ("deg", -45, 45), "Shock": ("N", 0, 5000),
    "Roll": ("deg", -10, 10), "Pitch": ("deg", -10, 10), "Yaw": ("deg", -10, 10),
    "Stiffness": ("N/mm", 0, 100), "Strut": ("mm", 0, 500), "ControlArm": ("mm", 0, 500),
    "SwayBar": ("N", 0, 5000), "SuspensionLink": ("mm", 0, 500), "SpringRate": ("N/mm", 0, 100),
    "Dampener": ("N", 0, 5000), "AirSpring": ("bar", 0, 10), "ShockAbsorber": ("N", 0, 5000),
    "SuspensionSensor": ("V", 0, 5), "RollBar": ("N", 0, 5000), "PitchAngle": ("deg", -10, 10),
    "YawRate": ("deg/s", 0, 100), "StiffnessControl": ("N/mm", 0, 100), "SuspensionHeight": ("mm", 0, 200),
    "LoadSensor": ("kg", 0, 2000), "StrutLength": ("mm", 0, 500), "ArmAngle": ("deg", -45, 45),
    "BarStress": ("MPa", 0, 500), "LinkWear": ("%", 0, 100), "DampRate": ("N/s", 0, 5000),
    "AirPressure": ("bar", 0, 10), "ShockTemp": ("°C", 0, 100), "SuspHeight": ("mm", 0, 200),
    "DampForce": ("N", 0, 5000), "SpringForce": ("N/mm", 0, 100), "LoadLevel": ("kg", 0, 2000),
    "AnglePos": ("deg", -45, 45), "ShockForce": ("N", 0, 5000), "RollAngle": ("deg", -10, 10),
    "PitchPos": ("deg", -10, 10), "YawAngle": ("deg", -10, 10), "StiffLevel": ("N/mm", 0, 100),
    "StrutPos": ("mm", 0, 500), "ArmPos": ("mm", 0, 500), "SwayForce": ("N", 0, 5000),
    "LinkPos": ("mm", 0, 500), "SpringLevel": ("N/mm", 0, 100), "DampLevel": ("N", 0, 5000),
    "AirSpringPress": ("bar", 0, 10), "ShockLevel": ("N", 0, 5000), "SuspSens": ("V", 0, 5),
    "RollForce": ("N", 0, 5000), "PitchRate": ("deg/s", 0, 100), "YawSpeed": ("deg/s", 0, 100),
    "StiffAdjust": ("N/mm", 0, 100), "HeightAdjust": ("mm", 0, 200),

    # Emissions category (56 signals)
    "CO2": ("g/km", 0, 200), "NOx": ("g/km", 0, 50), "PM": ("mg/km", 0, 10),
    "O2": ("%", 0, 21), "CO": ("g/km", 0, 10), "HC": ("g/km", 0, 5),
    "Lambda": ("ratio", 0.8, 1.2), "EGR": ("%", 0, 50), "DPF": ("g", 0, 10),
    "SCR": ("%", 0, 100), "EmissionSensor": ("V", 0, 5), "CatalystTemp": ("°C", 200, 1000),
    "ExhaustGas": ("°C", 300, 800), "ParticulateFilter": ("g", 0, 10), "NOxSensor": ("V", 0, 5),
    "LambdaProbe": ("V", 0, 5), "EGRValvePos": ("mm", 0, 20), "SCRFluid": ("L", 0, 20),
    "EmissionControl": ("bool", 0, 1), "ExhaustFlow": ("L/s", 0, 500), "CO2Level": ("ppm", 0, 1000),
    "HCLevel": ("ppm", 0, 500), "PMFilter": ("g", 0, 10), "O2Sensor": ("V", 0, 5),
    "SCRRate": ("L/h", 0, 10), "EmissionRate": ("g/s", 0, 1), "CatalystEff": ("%", 0, 100),
    "FilterLoad": ("g", 0, 10), "NOxRate": ("g/s", 0, 0.1), "ProbeVolt": ("V", 0, 5),
    "ValveFlow": ("L/s", 0, 10), "FluidLevel": ("mm", 0, 200), "CO2Emission": ("g/km", 0, 200),
    "NOxEmission": ("g/km", 0, 50), "PMLevel": ("mg/km", 0, 10), "O2Level": ("%", 0, 21),
    "COEmission": ("g/km", 0, 10), "HCEmision": ("g/km", 0, 5), "LambdaRatio": ("ratio", 0.8, 1.2),
    "EGRRate": ("%", 0, 50), "DPFLevel": ("g", 0, 10), "SCRLevel": ("%", 0, 100),
    "EmissSens": ("V", 0, 5), "CatTemp": ("°C", 200, 1000), "ExGasTemp": ("°C", 300, 800),
    "PartFilter": ("g", 0, 10), "NOxSens": ("V", 0, 5), "LambdaSens": ("V", 0, 5),
    "EGRPos": ("mm", 0, 20), "SCRFluidLevel": ("L", 0, 20), "EmissCtrl": ("bool", 0, 1),
    "ExFlow": ("L/s", 0, 500), "CO2Rate": ("ppm/s", 0, 100), "HCRate": ("ppm/s", 0, 50),
    "PMFiltration": ("g", 0, 10), "O2Sens": ("V", 0, 5), "SCRFlow": ("L/h", 0, 10),

    # Driver category (56 signals)
    "Throttle": ("%", 0, 100), "Brake": ("%", 0, 100), "Steering": ("deg", -450, 450),
    "Accel": ("m/s^2", -5, 5), "Turn": ("deg", -45, 45), "Clutch": ("%", 0, 100),
    "Input": ("%", 0, 100), "LaneDeparture": ("bool", 0, 1), "NightPedestrian": ("bool", 0, 1),
    "Drowsiness": ("bool", 0, 1), "ActivePark": ("bool", 0, 1), "BlindSpot": ("bool", 0, 1),
    "CrossTraffic": ("bool", 0, 1), "SteeringAngle": ("deg", -450, 450), "DriverMonitor": ("bool", 0, 1),
    "PedalPosition": ("%", 0, 100), "SteeringWheel": ("deg", -450, 450), "TurnSignal": ("bool", 0, 1),
    "HeadlightControl": ("bool", 0, 1), "WiperControl": ("bool", 0, 1), "Horn": ("bool", 0, 1),
    "EmergencyStop": ("bool", 0, 1), "DriverAlert": ("bool", 0, 1), "LaneKeep": ("bool", 0, 1),
    "TrafficSign": ("bool", 0, 1), "PedestrianDetect": ("bool", 0, 1), "BlindSpotAssist": ("bool", 0, 1),
    "SteerTorque": ("Nm", 0, 100), "PedalForce": ("N", 0, 500), "AlertLevel": ("%", 0, 100),
    "TrafficSpeed": ("km/h", 0, 250), "PedDetectRange": ("m", 0, 50), "BlindAssistStatus": ("bool", 0, 1),
    "ThrottlePos": ("%", 0, 100), "BrakePos": ("%", 0, 100), "SteerAngle": ("deg", -450, 450),
    "AccelRate": ("m/s^2", -5, 5), "TurnRate": ("deg/s", -45, 45), "ClutchPos": ("%", 0, 100),
    "LaneDepart": ("bool", 0, 1), "NightPed": ("bool", 0, 1), "DrowsyLevel": ("bool", 0, 1),
    "ParkAssist": ("bool", 0, 1), "BlindSpotSens": ("bool", 0, 1), "CrossTrafficSens": ("bool", 0, 1),
    "SteerWheel": ("deg", -450, 450), "PedalForceLevel": ("N", 0, 500), "AlertStatus": ("bool", 0, 1),
    "TrafficSignDetect": ("bool", 0, 1), "ThrottleLevel": ("%", 0, 100), "BrakeLevel": ("%", 0, 100),
    "SteerLevel": ("deg", -450, 450), "AccelLevel": ("m/s^2", -5, 5), "TurnLevel": ("deg", -45, 45),
    "ClutchLevel": ("%", 0, 100), "InputRate": ("%/s", 0, 100),

    # Environment category (56 signals)
    "AmbientTemp": ("°C", -20, 50), "Humidity": ("%", 0, 100), "Pressure": ("hPa", 800, 1200),
    "Altitude": ("m", 0, 3000), "Rain": ("mm/h", 0, 50), "Wind": ("km/h", 0, 100),
    "Light": ("lux", 0, 100000), "Noise": ("dB", 0, 100), "RoadTemp": ("°C", -20, 60),
    "AirQuality": ("index", 0, 500), "WeatherSensor": ("V", 0, 5), "FogLevel": ("%", 0, 100),
    "SnowDepth": ("mm", 0, 500), "IceDetect": ("bool", 0, 1), "WindDirection": ("deg", 0, 360),
    "SunIntensity": ("W/m^2", 0, 1000), "UVIndex": ("index", 0, 15), "Pollution": ("ppm", 0, 1000),
    "RoadCondition": ("bool", 0, 1), "AmbientNoise": ("dB", 0, 100), "RainRate": ("mm/h", 0, 50),
    "HumidityLevel": ("%", 0, 100), "AltitudeChange": ("m/s", -10, 10), "AirPollution": ("ppm", 0, 1000),
    "LightSensor": ("lux", 0, 100000), "FogDensity": ("%", 0, 100), "SnowRate": ("mm/h", 0, 50),
    "IceThickness": ("mm", 0, 50), "WindSpeed": ("km/h", 0, 100), "SunAngle": ("deg", 0, 90),
    "UVExposure": ("index", 0, 15), "PollutionSource": ("ppm", 0, 1000), "AmbientHeat": ("°C", -20, 50),
    "HumidLevel": ("%", 0, 100), "PressLevel": ("hPa", 800, 1200), "AltLevel": ("m", 0, 3000),
    "RainLevel": ("mm/h", 0, 50), "WindLevel": ("km/h", 0, 100), "LightLevel": ("lux", 0, 100000),
    "NoiseLevel": ("dB", 0, 100), "RoadHeat": ("°C", -20, 60), "AirQual": ("index", 0, 500),
    "WeatherSens": ("V", 0, 5), "FogRate": ("%", 0, 100), "SnowLevel": ("mm", 0, 500),
    "IceLevel": ("bool", 0, 1), "WindDir": ("deg", 0, 360), "SunLevel": ("W/m^2", 0, 1000),
    "UVLevel": ("index", 0, 15), "PollutionLevel": ("ppm", 0, 1000), "RoadCond": ("bool", 0, 1),
    "AmbientSound": ("dB", 0, 100), "RainFlow": ("mm/h", 0, 50), "HumidRate": ("%/h", 0, 100),
    "AltChange": ("m/s", -10, 10), "AirPollute": ("ppm", 0, 1000), "LightSens": ("lux", 0, 100000),

    # Vision category (56 signals)
    "NightVision": ("bool", 0, 1), "ThermalVision": ("bool", 0, 1), "InfraredCamera": ("bool", 0, 1),
    "DayVision": ("bool", 0, 1), "WideAngleVision": ("bool", 0, 1), "RearVision": ("bool", 0, 1),
    "SideVision": ("bool", 0, 1), "ForwardVision": ("bool", 0, 1), "ObjectRecognition": ("bool", 0, 1),
    "LaneVision": ("bool", 0, 1), "PedestrianVision": ("bool", 0, 1), "TrafficVision": ("bool", 0, 1),
    "NightIR": ("bool", 0, 1), "DayIR": ("bool", 0, 1), "VisionProcessor": ("Hz", 0, 5000),
    "VisionSensor": ("V", 0, 5), "WideField": ("deg", 0, 180), "RearField": ("deg", 0, 180),
    "SideField": ("deg", 0, 180), "ForwardField": ("deg", 0, 180), "ObjectDist": ("m", 0, 200),
    "LaneWidth": ("m", 0, 5), "PedHeight": ("m", 0, 2), "TrafficCount": ("count", 0, 100),
    "IRTemp": ("°C", -20, 50), "ProcLoad": ("%", 0, 100), "SensorRange": ("m", 0, 200),
    "FieldDepth": ("m", 0, 100), "RearObj": ("m", 0, 10), "SideObj": ("m", 0, 10),
    "NightVis": ("bool", 0, 1), "ThermalVis": ("bool", 0, 1), "InfraCam": ("bool", 0, 1),
    "DayVis": ("bool", 0, 1), "WideVis": ("bool", 0, 1), "RearVis": ("bool", 0, 1),
    "SideVis": ("bool", 0, 1), "FwdVis": ("bool", 0, 1), "ObjRecog": ("bool", 0, 1),
    "LaneVis": ("bool", 0, 1), "PedVis": ("bool", 0, 1), "TrafficVis": ("bool", 0, 1),
    "NightInfra": ("bool", 0, 1), "DayInfra": ("bool", 0, 1), "VisProc": ("Hz", 0, 5000),
    "VisSens": ("V", 0, 5), "WideAngle": ("deg", 0, 180), "RearAngle": ("deg", 0, 180),
    "SideAngle": ("deg", 0, 180), "FwdAngle": ("deg", 0, 180), "ObjDistance": ("m", 0, 200),
    "LaneSize": ("m", 0, 5), "PedSize": ("m", 0, 2), "TrafficVolume": ("count", 0, 100),
    "IRHeat": ("°C", -20, 50), "ProcUsage": ("%", 0, 100),

    # Camera category (56 signals)
    "FrontObjectCCD": ("bool", 0, 1), "RearObjectCCD": ("bool", 0, 1), "Rear": ("bool", 0, 1),
    "SideCamera": ("bool", 0, 1), "InteriorCamera": ("bool", 0, 1), "DashboardCamera": ("bool", 0, 1),
    "RoofCamera": ("bool", 0, 1), "UnderCarCamera": ("bool", 0, 1), "360Camera": ("bool", 0, 1),
    "StereoCamera": ("bool", 0, 1), "ThermalCamera": ("bool", 0, 1), "NightCamera": ("bool", 0, 1),
    "WideCamera": ("bool", 0, 1), "ZoomCamera": ("bool", 0, 1), "ParkingCamera": ("bool", 0, 1),
    "BlindSpotCamera": ("bool", 0, 1), "CrossTrafficCamera": ("bool", 0, 1), "LaneCamera": ("bool", 0, 1),
    "TrafficCamera": ("bool", 0, 1), "ObjectCamera": ("bool", 0, 1), "FrontZoom": ("bool", 0, 1),
    "RearPan": ("deg", 0, 180), "SideTilt": ("deg", -45, 45), "IntFocus": ("bool", 0, 1),
    "DashAngle": ("deg", 0, 45), "RoofView": ("bool", 0, 1), "UnderScan": ("bool", 0, 1),
    "StereoDepth": ("m", 0, 10), "ThermalRange": ("m", 0, 50), "NightGain": ("%", 0, 100),
    "FrontCam": ("bool", 0, 1), "RearCam": ("bool", 0, 1), "SideCam": ("bool", 0, 1),
    "IntCam": ("bool", 0, 1), "DashCam": ("bool", 0, 1), "RoofCam": ("bool", 0, 1),
    "UnderCam": ("bool", 0, 1), "360Cam": ("bool", 0, 1), "StereoCam": ("bool", 0, 1),
    "ThermalCam": ("bool", 0, 1), "NightCam": ("bool", 0, 1), "WideCam": ("bool", 0, 1),
    "ZoomCam": ("bool", 0, 1), "ParkCam": ("bool", 0, 1), "BlindCam": ("bool", 0, 1),
    "CrossCam": ("bool", 0, 1), "LaneCam": ("bool", 0, 1), "TrafficCam": ("bool", 0, 1),
    "ObjCam": ("bool", 0, 1), "FrontFocus": ("bool", 0, 1), "RearTilt": ("deg", -45, 45),
    "SideAngleCam": ("deg", 0, 180), "IntZoom": ("bool", 0, 1), "DashView": ("bool", 0, 1),
    "RoofScan": ("bool", 0, 1),

    # Airbag category (56 signals)
    "FrontSensors": ("bool", 0, 1), "SideCurtain": ("bool", 0, 1), "CollisionSRS": ("bool", 0, 1),
    "SideSRS": ("bool", 0, 1), "RearAirbag": ("bool", 0, 1), "RoofAirbag": ("bool", 0, 1),
    "KneeAirbag": ("bool", 0, 1), "PedestrianAirbag": ("bool", 0, 1), "AirbagSensor": ("V", 0, 5),
    "AirbagControl": ("bool", 0, 1), "SideImpact": ("bool", 0, 1), "FrontImpact": ("bool", 0, 1),
    "RearImpact": ("bool", 0, 1), "RoofImpact": ("bool", 0, 1), "KneeImpact": ("bool", 0, 1),
    "PedestrianImpact": ("bool", 0, 1), "AirbagDeploy": ("bool", 0, 1), "AirbagStatus": ("bool", 0, 1),
    "AirbagPressure": ("bar", 0, 10), "AirbagTimer": ("s", 0, 10), "SensorForce": ("N", 0, 1000),
    "CurtainSpeed": ("m/s", 0, 20), "SRSPower": ("V", 0, 24), "ImpactAngle": ("deg", -90, 90),
    "DeployTime": ("ms", 0, 100), "StatusCode": ("code", 0, 255), "PressureRate": ("bar/s", 0, 10),
    "TimerReset": ("s", 0, 10), "FrontSens": ("bool", 0, 1), "SideAirbag": ("bool", 0, 1),
    "CollSRS": ("bool", 0, 1), "SideBag": ("bool", 0, 1), "RearBag": ("bool", 0, 1),
    "RoofBag": ("bool", 0, 1), "KneeBag": ("bool", 0, 1), "PedBag": ("bool", 0, 1),
    "AirbagSens": ("V", 0, 5), "AirbagCtrl": ("bool", 0, 1), "SideHit": ("bool", 0, 1),
    "FrontHit": ("bool", 0, 1), "RearHit": ("bool", 0, 1), "RoofHit": ("bool", 0, 1),
    "KneeHit": ("bool", 0, 1), "PedHit": ("bool", 0, 1), "AirbagAct": ("bool", 0, 1),
    "BagStatus": ("bool", 0, 1), "BagPress": ("bar", 0, 10), "BagTimer": ("s", 0, 10),
    "ForceSens": ("N", 0, 1000), "CurtainRate": ("m/s", 0, 20), "SRSEnergy": ("V", 0, 24),
    "ImpactPos": ("deg", -90, 90), "DeployRate": ("ms", 0, 100), "StatusLevel": ("code", 0, 255),

    # Radar category (56 signals)
    "FrontObjectLaser": ("m", 0, 200), "RearObjectLaser": ("m", 0, 200), "SideRadar": ("m", 0, 100),
    "CornerRadar": ("m", 0, 100), "LongRangeRadar": ("m", 0, 200), "ShortRangeRadar": ("m", 0, 50),
    "BlindSpotRadar": ("m", 0, 10), "CrossTrafficRadar": ("m", 0, 20), "ParkingRadar": ("m", 0, 10),
    "AdaptiveRadar": ("m", 0, 200), "DistanceRadar": ("m", 0, 200), "SpeedRadar": ("km/h", 0, 250),
    "ObjectRadar": ("m", 0, 100), "LaneRadar": ("m", 0, 10), "TrafficRadar": ("km/h", 0, 250),
    "CollisionRadar": ("m", 0, 10), "ObstacleRadar": ("m", 0, 10), "RangeSensor": ("m", 0, 200),
    "RadarBeam": ("deg", 0, 90), "RadarPulse": ("Hz", 0, 10000), "LaserPower": ("W", 0, 10),
    "RadarAngle": ("deg", 0, 90), "ShortRange": ("m", 0, 50), "BlindDist": ("m", 0, 10),
    "CrossAngle": ("deg", 0, 45), "ParkDist": ("m", 0, 10), "AdaptSpeed": ("km/h", 0, 200),
    "TrafficFlow": ("veh/h", 0, 1000), "ObstacleSize": ("m", 0, 5), "PulseRate": ("Hz", 0, 10000),
    "FrontLaser": ("m", 0, 200), "RearLaser": ("m", 0, 200), "SideScan": ("m", 0, 100),
    "CornerScan": ("m", 0, 100), "LongRadar": ("m", 0, 200), "ShortRadar": ("m", 0, 50),
    "BlindRadar": ("m", 0, 10), "CrossRadar": ("m", 0, 20), "ParkRadar": ("m", 0, 10),
    "AdaptRadar": ("m", 0, 200), "DistRadar": ("m", 0, 200), "SpeedScan": ("km/h", 0, 250),
    "ObjRadar": ("m", 0, 100), "LaneScan": ("m", 0, 10), "TrafficScan": ("km/h", 0, 250),
    "CollRadar": ("m", 0, 10), "ObstRadar": ("m", 0, 10), "RangeScan": ("m", 0, 200),
    "BeamAngle": ("deg", 0, 90), "PulseFreq": ("Hz", 0, 10000), "LaserEnergy": ("W", 0, 10),
    "RadarPos": ("deg", 0, 90), "ShortDist": ("m", 0, 50), "BlindRange": ("m", 0, 10),
    "CrossDist": ("m", 0, 20), "ParkRange": ("m", 0, 10),
    
    # Control category (56 signalsl) 
    "CentralComputer": ("bool", 0, 1), "ECUControl": ("bool", 0, 1), "TCUControl": ("bool", 0, 1),
    "ABSControl": ("bool", 0, 1), "TractionControl": ("bool", 0, 1), "StabilityControl": ("bool", 0, 1),
    "SuspensionControl": ("bool", 0, 1), "BrakeControl": ("bool", 0, 1), "EngineControl": ("bool", 0, 1),
    "TransmissionControl": ("bool", 0, 1), "AirbagControl": ("bool", 0, 1), "RadarControl": ("bool", 0, 1),
    "CameraControl": ("bool", 0, 1), "VisionControl": ("bool", 0, 1), "DriverAssist": ("bool", 0, 1),
    "InfotainmentControl": ("bool", 0, 1), "ClimateControl": ("bool", 0, 1), "LightingControl": ("bool", 0, 1),
    "SecurityControl": ("bool", 0, 1), "DiagnosticsControl": ("bool", 0, 1), "CompStatus": ("bool", 0, 1),
    "ECUError": ("code", 0, 255), "TCUMode": ("mode", 0, 5), "ABSRate": ("bar/s", 0, 10),
    "TractionLevel": ("%", 0, 100), "StabilityAngle": ("deg", -10, 10), "SuspMode": ("mode", 0, 3),
    "BrakeMode": ("mode", 0, 3), "EngMode": ("mode", 0, 3), "TransMode": ("mode", 0, 3),
    "CompCtrl": ("bool", 0, 1), "ECUCtrl": ("bool", 0, 1), "TCUCtrl": ("bool", 0, 1),
    "ABSCtrl": ("bool", 0, 1), "TractionCtrl": ("bool", 0, 1), "StabCtrl": ("bool", 0, 1),
    "SuspCtrl": ("bool", 0, 1), "BrakeCtrl": ("bool", 0, 1), "EngCtrl": ("bool", 0, 1),
    "TransCtrl": ("bool", 0, 1), "AirbagCtrl": ("bool", 0, 1), "RadarCtrl": ("bool", 0, 1),
    "CamCtrl": ("bool", 0, 1), "VisionCtrl": ("bool", 0, 1), "AssistCtrl": ("bool", 0, 1),
    "InfoCtrl": ("bool", 0, 1), "ClimateCtrl": ("bool", 0, 1), "LightCtrl": ("bool", 0, 1),
    "SecureCtrl": ("bool", 0, 1), "DiagCtrl": ("bool", 0, 1), "CompState": ("bool", 0, 1),
    "ECUStatus": ("bool", 0, 1), "TCUStatus": ("bool", 0, 1), "ABSLevel": ("bar", 0, 10),
    "TractionRate": ("%", 0, 100), "StabAngle": ("deg", -10, 10), "SuspAdjust": ("mm", 0, 200),
    "BrakeAdjust": ("bar", 0, 10), "EngAdjust": ("rpm", 700, 7000), "TransAdjust": ("ratio", 0, 5),

    # Chassis category (56 signals)
    "Frame": ("mm", 0, 5000), "ChassisRail": ("mm", 0, 5000), "CrossMember": ("mm", 0, 1000),
    "Subframe": ("mm", 0, 1000), "MountingPoint": ("mm", 0, 500), "ChassisSensor": ("V", 0, 5),
    "Alignment": ("deg", -5, 5), "TorsionBar": ("Nm", 0, 1000), "Stabilizer": ("N", 0, 5000),
    "ChassisLoad": ("kg", 0, 2000), "FrameStrength": ("MPa", 0, 1000), "RailStress": ("MPa", 0, 500),
    "CrossStress": ("MPa", 0, 500), "SubframeLoad": ("kg", 0, 1000), "MountStress": ("MPa", 0, 500),
    "AlignmentSensor": ("V", 0, 5), "TorsionSensor": ("Nm", 0, 1000), "StabilizerBar": ("N", 0, 5000),
    "ChassisFlex": ("mm", 0, 50), "FrameTwist": ("deg", 0, 5), "RailBend": ("mm", 0, 50),
    "CrossDeflect": ("mm", 0, 50), "SubLoadDist": ("kg", 0, 1000), "MountWear": ("%", 0, 100),
    "AlignError": ("deg", -5, 5), "TorsionAngle": ("deg", 0, 10), "StabForce": ("N", 0, 5000),
    "FlexRate": ("mm/s", 0, 10), "TwistLimit": ("deg", 0, 5), "FrameLoad": ("kg", 0, 2000),
    "RailLoad": ("kg", 0, 1000), "CrossLoad": ("kg", 0, 1000), "SubframeStress": ("MPa", 0, 500),
    "MountLoad": ("kg", 0, 500), "ChassisSens": ("V", 0, 5), "AlignPos": ("deg", -5, 5),
    "TorsionLoad": ("Nm", 0, 1000), "StabLoad": ("N", 0, 5000), "ChassisWeight": ("kg", 0, 2000),
    "FrameForce": ("N", 0, 10000), "RailForce": ("N", 0, 5000), "CrossForce": ("N", 0, 5000),
    "SubLoad": ("kg", 0, 1000), "MountForce": ("N", 0, 500), "AlignSens": ("V", 0, 5),
    "TorsionSens": ("Nm", 0, 1000), "StabBar": ("N", 0, 5000), "ChassisBend": ("mm", 0, 50),
    "FrameAngle": ("deg", 0, 5), "RailAngle": ("deg", 0, 5), "CrossAngle": ("deg", 0, 5),
    "SubStress": ("MPa", 0, 500), "MountAngle": ("deg", 0, 5), "AlignAdjust": ("deg", -5, 5),

    # Infotainment category (56 signals)
    "Display": ("bool", 0, 1), "Touchscreen": ("bool", 0, 1), "Audio": ("bool", 0, 1),
    "Navigation": ("bool", 0, 1), "Bluetooth": ("bool", 0, 1), "WiFi": ("bool", 0, 1),
    "USB": ("bool", 0, 1), "HDMI": ("bool", 0, 1), "Speaker": ("bool", 0, 1),
    "Microphone": ("bool", 0, 1), "InfotainmentCPU": ("Hz", 0, 5000), "GPS": ("bool", 0, 1),
    "Radio": ("bool", 0, 1), "Satellite": ("bool", 0, 1), "MediaPlayer": ("bool", 0, 1),
    "VoiceControl": ("bool", 0, 1), "GestureControl": ("bool", 0, 1), "ScreenBrightness": ("%", 0, 100),
    "AudioVolume": ("%", 0, 100), "NavigationMap": ("bool", 0, 1), "DisplayRes": ("px", 0, 1920),
    "TouchSens": ("V", 0, 5), "AudioFreq": ("Hz", 20, 20000), "NavRoute": ("km", 0, 1000),
    "BTSignal": ("dBm", -100, 0), "WiFiStrength": ("dBm", -100, 0), "USBSpeed": ("Mbps", 0, 5000),
    "HDMIPort": ("bool", 0, 1), "SpeakerLoc": ("deg", 0, 360), "MicGain": ("dB", 0, 50),
    "DisplaySize": ("inch", 0, 15), "TouchResponse": ("ms", 0, 100), "AudioLevel": ("%", 0, 100),
    "NavPath": ("km", 0, 1000), "BTConnect": ("bool", 0, 1), "WiFiConnect": ("bool", 0, 1),
    "USBPort": ("bool", 0, 1), "HDMIConnect": ("bool", 0, 1), "SpeakerVol": ("%", 0, 100),
    "MicLevel": ("dB", 0, 50), "InfoCPU": ("Hz", 0, 5000), "GPSLoc": ("bool", 0, 1),
    "RadioFreq": ("MHz", 87.5, 108), "SatSignal": ("dBm", -100, 0), "MediaTrack": ("count", 0, 100),
    "VoiceCmd": ("bool", 0, 1), "GestureCmd": ("bool", 0, 1), "ScreenLight": ("lux", 0, 500),
    "AudioOutput": ("W", 0, 50), "NavMap": ("bool", 0, 1), "DisplayMode": ("mode", 0, 3),
    "TouchMode": ("mode", 0, 3), "AudioMode": ("mode", 0, 3), "NavMode": ("mode", 0, 3),
    "BTMode": ("mode", 0, 3), "WiFiMode": ("mode", 0, 3),

    # BatteryManagement category (56 signals)
    "BatteryVoltage": ("V", 0, 24), "BatteryCurrent": ("A", -200, 200), "BatteryTemp": ("°C", 0, 60),
    "ChargeLevel": ("%", 0, 100), "StateOfCharge": ("%", 0, 100), "StateOfHealth": ("%", 0, 100),
    "CellVoltage": ("V", 0, 4.2), "CellTemp": ("°C", 0, 60), "PackVoltage": ("V", 0, 400),
    "PackCurrent": ("A", -200, 200), "CoolantFlow": ("L/min", 0, 20), "ThermalRunaway": ("°C", 0, 1000),
    "BatterySensor": ("V", 0, 5), "ChargeRate": ("A", 0, 200), "DischargeRate": ("A", 0, 200),
    "BatteryBalance": ("mV", -50, 50), "CellMonitor": ("V", 0, 4.2), "PackMonitor": ("V", 0, 400),
    "ThermalSensor": ("°C", 0, 60), "ChargeCycle": ("count", 0, 1000), "CellBal": ("mV", -50, 50),
    "PackTemp": ("°C", 0, 60), "CoolFlowRate": ("L/min", 0, 20), "RunawayTemp": ("°C", 0, 1000),
    "SensorVolt": ("V", 0, 5), "ChargeEff": ("%", 0, 100), "DischargeCap": ("Ah", 0, 100),
    "BalStatus": ("bool", 0, 1), "MonAccuracy": ("mV", 0, 10), "CycleCount": ("count", 0, 1000),
    "BattVolt": ("V", 0, 24), "BattCurrent": ("A", -200, 200), "BattHeat": ("°C", 0, 60),
    "ChargeLev": ("%", 0, 100), "SOCLevel": ("%", 0, 100), "SOHLevel": ("%", 0, 100),
    "CellVolt": ("V", 0, 4.2), "CellHeat": ("°C", 0, 60), "PackVolt": ("V", 0, 400),
    "PackCurrent": ("A", -200, 200), "CoolRate": ("L/min", 0, 20), "ThermalRisk": ("°C", 0, 1000),
    "BattSens": ("V", 0, 5), "ChargeSpeed": ("A", 0, 200), "DischargeSpeed": ("A", 0, 200),
    "BattBal": ("mV", -50, 50), "CellMon": ("V", 0, 4.2), "PackMon": ("V", 0, 400),
    "ThermalSens": ("°C", 0, 60), "CycleNum": ("count", 0, 1000), "CellBalance": ("mV", -50, 50),
    "PackHeat": ("°C", 0, 60), "CoolFlow": ("L/min", 0, 20), "RunawayHeat": ("°C", 0, 1000),
    "SensorVoltage": ("V", 0, 5), "ChargeEfficiency": ("%", 0, 100),

    # Switch signals (90 signals from switch_signals)
    "Headlight": ("bool", 0, 1), "Wiper": ("bool", 0, 1), "Ignition": ("bool", 0, 1),
    "TurnSignal": ("bool", 0, 1), "ParkingBrake": ("bool", 0, 1), "EngineStart": ("bool", 0, 1),
    "AC": ("bool", 0, 1), "FogLight": ("bool", 0, 1), "HighBeam": ("bool", 0, 1),
    "LowBeam": ("bool", 0, 1), "BrakeLight": ("bool", 0, 1), "HazardLight": ("bool", 0, 1),
    "InteriorLight": ("bool", 0, 1), "TrunkLight": ("bool", 0, 1), "DomeLight": ("bool", 0, 1),
    "MirrorHeater": ("bool", 0, 1), "SeatHeater": ("bool", 0, 1), "Defroster": ("bool", 0, 1),
    "RearDefogger": ("bool", 0, 1), "Sunroof": ("bool", 0, 1), "PowerWindow": ("bool", 0, 1),
    "DoorLock": ("bool", 0, 1), "TrunkRelease": ("bool", 0, 1), "FuelCapRelease": ("bool", 0, 1),
    "HoodRelease": ("bool", 0, 1), "HornSwitch": ("bool", 0, 1), "CruiseControlSwitch": ("bool", 0, 1),
    "TractionControlSwitch": ("bool", 0, 1), "StabilityControlSwitch": ("bool", 0, 1), "LaneKeepAssist": ("bool", 0, 1),
    "BlindSpotMonitor": ("bool", 0, 1), "AdaptiveCruiseSwitch": ("bool", 0, 1), "EmergencyBrakeAssist": ("bool", 0, 1),
    "HillStartAssist": ("bool", 0, 1), "AutoHold": ("bool", 0, 1), "EcoMode": ("bool", 0, 1),
    "SportMode": ("bool", 0, 1), "SnowMode": ("bool", 0, 1), "OffRoadMode": ("bool", 0, 1),
    "FourWheelDrive": ("bool", 0, 1), "TowMode": ("bool", 0, 1), "EngineFan": ("bool", 0, 1),
    "RadiatorFan": ("bool", 0, 1), "Heater": ("bool", 0, 1), "Ventilation": ("bool", 0, 1),
    "AirPurifier": ("bool", 0, 1), "ChildLock": ("bool", 0, 1), "PowerMirror": ("bool", 0, 1),
    "SteeringWheelHeater": ("bool", 0, 1), "AlarmSystem": ("bool", 0, 1), "HeadlightAuto": ("bool", 0, 1),
    "WiperAuto": ("bool", 0, 1), "IgnitionAuto": ("bool", 0, 1), "TurnSignalAuto": ("bool", 0, 1),
    "BrakeAuto": ("bool", 0, 1), "EngineStop": ("bool", 0, 1), "ACAuto": ("bool", 0, 1),
    "FogLightAuto": ("bool", 0, 1), "HighBeamAuto": ("bool", 0, 1), "LowBeamAuto": ("bool", 0, 1),
    "BrakeLightAuto": ("bool", 0, 1), "HazardAuto": ("bool", 0, 1), "InteriorLightAuto": ("bool", 0, 1),
    "TrunkLightAuto": ("bool", 0, 1), "DomeLightAuto": ("bool", 0, 1), "MirrorHeaterAuto": ("bool", 0, 1),
    "SeatHeaterAuto": ("bool", 0, 1), "DefrosterAuto": ("bool", 0, 1), "RearDefoggerAuto": ("bool", 0, 1),
    "SunroofAuto": ("bool", 0, 1), "WindowAuto": ("bool", 0, 1), "DoorLockAuto": ("bool", 0, 1),
    "TrunkReleaseAuto": ("bool", 0, 1), "FuelCapAuto": ("bool", 0, 1), "HoodReleaseAuto": ("bool", 0, 1),
    "HornAuto": ("bool", 0, 1), "CruiseAuto": ("bool", 0, 1), "TractionAuto": ("bool", 0, 1),
    "StabilityAuto": ("bool", 0, 1), "LaneAssistAuto": ("bool", 0, 1), "BlindSpotAuto": ("bool", 0, 1),
    "CruiseAdaptAuto": ("bool", 0, 1), "EmergencyBrakeAuto": ("bool", 0, 1), "HillAssistAuto": ("bool", 0, 1),
    "AutoHoldSwitch": ("bool", 0, 1), "EcoModeSwitch": ("bool", 0, 1), "SportModeSwitch": ("bool", 0, 1),
    "SnowModeSwitch": ("bool", 0, 1), "OffRoadSwitch": ("bool", 0, 1), "FourWheelSwitch": ("bool", 0, 1),
    "TowModeSwitch": ("bool", 0, 1), "EngineFanAuto": ("bool", 0, 1), "RadiatorFanAuto": ("bool", 0, 1),
    "HeaterAuto": ("bool", 0, 1), "VentilationAuto": ("bool", 0, 1), "AirPurifierAuto": ("bool", 0, 1),
    "ChildLockAuto": ("bool", 0, 1), "PowerMirrorAuto": ("bool", 0, 1), "SteeringHeaterAuto": ("bool", 0, 1),
    "AlarmAuto": ("bool", 0, 1)
}

In [ ]:
# ---------------- Helper Functions ----------------

def get_unique_signals(cat, sw):
    """
    Extract a list of unique signal names from the provided categories and switch signals.
    Prints any duplicate names found across categories.

    Returns:
        unique (list): All unique signal names.
        overlaps (list): List of switch signals that overlap with numeric signals.
    """
    all_cat = [n for names in cat.values() for n in names]
    dups = [item for item, cnt in Counter(all_cat).items() if cnt > 1]
    if dups:
        for d in dups:
            #print(f"'{d}' in: {', '.join(c for c, names in cat.items() if d in names)}")
    unique = list(dict.fromkeys(all_cat))
    unique += [n for n in sw if n not in unique]
    overlaps = [n for n in sw if n in unique]
    return unique, overlaps

def gen_num_data(n, rng, dt):
    """
    Generate numeric data with n samples within the specified range and data type.

    Parameters:
        n (int): Number of samples.
        rng (tuple): (min, max) range.
        dt (str): Data type ("float64", "int64", or "uint64").

    Returns:
        np.ndarray: Generated numeric data.
    """
    mn, mx = rng
    if dt == "float64":
        return np.random.uniform(mn, mx, n).astype(np.float64)
    elif dt == "int64":
        return np.random.randint(int(mn), int(mx) + 1, n).astype(np.int64)
    elif dt == "uint64":
        return np.random.randint(max(0, int(mn)), int(mx) + 1, n).astype(np.uint64)
    else:
        raise ValueError(f"Unknown dtype: {dt}")

def gen_switch_data(n):
    """
    Generate switch data (b'ON' or b'OFF') for n samples, stored as 'S2'.

    Parameters:
        n (int): Number of samples.

    Returns:
        np.ndarray: Array of switch values.
    """
    return np.random.choice([b'ON', b'OFF'], size=n).astype('S2')

def group_and_create_channels(ch_info, dur):
    """
    Group channel information by frequency and create MDF Signal objects for each channel.
    All channels in a frequency group share a common time base.

    Parameters:
        ch_info (list): List of tuples in the form:
            (name, freq, unit, (min, max), dtype, is_switch, force_obj)
        dur (int): Duration (in seconds).

    Returns:
        dict: Mapping from frequency to a list of Signal objects.
    """
    groups = {}
    for info in ch_info:
        freq = info[1]
        groups.setdefault(freq, []).append(info)

    grp_signals = {}
    for freq, infos in groups.items():
        n = int(dur * freq)
        t = np.linspace(0, dur, n, dtype=np.float64)
        sigs = []
        for name, _, unit, rng, dt, is_sw, force_obj in infos:
            if is_sw:
                data = gen_switch_data(n)
                sig = Signal(samples=data, timestamps=t, name=name, unit=unit, encoding='utf-8')
            elif force_obj:
                numeric = gen_num_data(n, rng, "float64")
                data = np.array([str(x).encode('utf-8') for x in numeric], dtype="S32")
                sig = Signal(samples=data, timestamps=t, name=name, unit=unit, encoding='utf-8')
            else:
                data = gen_num_data(n, rng, dt)
                sig = Signal(samples=data, timestamps=t, name=name, unit=unit)
            sigs.append(sig)
        grp_signals[freq] = sigs
    return grp_signals

# ---------------- Main Script ----------------

if __name__ == "__main__":
    # Create the base output folder.
    base_output_dir = "sample_data_set"
    os.makedirs(base_output_dir, exist_ok=True)

    # Get unique signals and print any overlaps.
    all_signals, overlaps = get_unique_signals(signal_categories, switch_signals)
    print(f"Switch signals overlapping with numeric signals: {overlaps}")
    print(f"Total unique signals available: {len(all_signals)}")

    # ---------------- Generation Parameters ----------------
    # Each MDF file will contain a random subset of the unique signals,
    # typically between 300 and 1200 channels (except the final file for a vehicle,
    # which uses all available signals). In addition, four extra speed channels are added.
    # The following parameters determine the data size:
    #
    #   duration = 600 seconds per file
    #   num_files = 12 files generated per vehicle
    #   possible_freq = [0.1, 1, 10, 100] Hz (sampling frequencies)
    #   vehicle_models = ["A", "B", "C"]
    #
    # With these parameters, the generated dataset is approximately 11GB in total.
    # You can adjust these values to generate a larger or smaller dataset as needed.
    duration = 600            # Duration in seconds for each file.
    num_files = 12            # Number of MDF files to generate per vehicle.
    possible_freq = [0.1, 1, 10, 100]   # Sampling frequencies.
    dtypes = ["float64", "int64", "uint64"]  # Data types for numeric signals.
    vehicle_models = ["A", "B", "C"]  # Dif vehicle models name.

    # Generate files for each vehicle model.
    for model in vehicle_models:
        model_folder = os.path.join(base_output_dir, f"VEH_MOD_{model}")
        os.makedirs(model_folder, exist_ok=True)

        # Create a random vehicle ID (e.g., A123456).
        vehicle_id = f"{model}{random.randint(0, 999999):06d}"
        vehicle_folder = os.path.join(model_folder, f"VEH_ID_{vehicle_id}")
        os.makedirs(vehicle_folder, exist_ok=True)

        for i in range(num_files):
            mdf = MDF(version="4.10")
            if i == num_files - 1:
                num_sig = len(all_signals)
                print(f"File {i+1}: Using ALL unique signals ({num_sig}).")
                selected = all_signals[:]
            else:
                num_sig = min(random.randint(300, 1200), len(all_signals))
                print(f"File {i+1}: Using a random subset of {num_sig} signals.")
                selected = random.sample(all_signals, num_sig)

            ch_info = []
            for name in selected:
                freq = random.choice(possible_freq)
                if name in switch_signals:
                    ch_info.append((name, freq, "", None, "object", True, True))
                else:
                    props = default_properties.get(name)
                    if not props:
                        # Determine category and apply fallback.
                        cat_found = None
                        for cat_key, cat_names in signal_categories.items():
                            if name in cat_names:
                                cat_found = cat_key
                                break
                        if cat_found == "Temperature":
                            props = ("°C", 20, 100)
                        elif cat_found == "Speed":
                            props = ("km/h", 0, 250)
                        elif cat_found == "Engine":
                            props = ("rpm", 700, 7000)
                        else:
                            props = ("unitless", 0, 1)
                    unit, mn, mx = props
                    force_obj = (random.random() < 0.5)
                    dt = "object" if force_obj else random.choice(dtypes)
                    ch_info.append((name, freq, unit, (mn, mx), dt, False, force_obj))

            # Always add the extra speed channels at 100 Hz.
            for extra_name in ["Speed_1", "speed_001", "speed", "SPEED"]:
                freq = 100
                unit, mn, mx = ("km/h", 0, 250)
                dt = random.choice(dtypes)
                ch_info.append((extra_name, freq, unit, (mn, mx), dt, False, False))

            groups = group_and_create_channels(ch_info, duration)
            for freq, sigs in groups.items():
                mdf.append(sigs, common_timebase=True)

            fname = os.path.join(vehicle_folder, f"VEH_MOD_{model}_VEH_ID_{vehicle_id}_file_{i+1}.mf4")
            mdf.save(fname, overwrite=True)
            print(f"Generated: {fname} | Channels used: {num_sig} + 4 extra speed channels")

    print("All files generated successfully.")


'Tire' in: Speed, Pressure
'Gear' in: Speed, Transmission
'Brake' in: Speed, Temperature, Pressure, Driver
'TransSpeed' in: Speed, Transmission
'TransRatio' in: Speed, Transmission
'Torque' in: Engine, Transmission
'Load' in: Engine, Transmission, Suspension
'Boost' in: Engine, Pressure
'Throttle' in: Engine, Driver
'ExhaustFlow' in: Engine, Emissions
'ThrottlePos' in: Engine, Driver
'TurboPressure' in: Engine, Pressure
'Coolant' in: Temperature, Pressure
'Oil' in: Temperature, Pressure
'Exhaust' in: Temperature, Pressure
'Fuel' in: Temperature, Pressure
'Battery' in: Temperature, Electrical
'TurboInlet' in: Temperature, Pressure
'CoolantFlow' in: Temperature, BatteryManagement
'ExhaustGas' in: Temperature, Emissions
'FuelLine' in: Temperature, Fuel
'HeaterTemp' in: Temperature, Fuel
'BrakeBooster' in: Pressure, Brake
'TransPress' in: Pressure, Transmission
'ClutchPress' in: Pressure, Transmission
'BattVolt' in: Electrical, BatteryManagement
'SensorVolt' in: Electrical, BatteryManageme

How to Read MF4 file

In [ ]:
with MDF('/sample_data_set/VEH_MOD_C/VEH_ID_C532070/VEH_MOD_C_VEH_ID_C532070_file_2.mf4') as mdf:
  df = mdf.to_dataframe(raster=1, use_display_names = True, time_as_date = True, only_basenames = True)
  df.info()
  
  mdf_info = mdf.info()
  print(mdf_info)

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 601 entries, 2025-03-28 15:07:03.319909+01:00 to 2025-03-28 15:17:03.319909+01:00
Columns: 780 entries, ProbeVolt to DiffSpeed
dtypes: float64(127), int64(133), object(399), uint64(121)
memory usage: 3.6+ MB
{'version': '4.10', 'program': 'amdf8.1.', 'comment': '<HDcomment>\n <TX/>\n <common_properties/>\n</HDcomment>', 'groups': 4, 'group 0': {'cycles': 600, 'comment': 'Python', 'channels count': 185, 'channel 0': 'name="time" type=MASTER', 'channel 1': 'name="ParkRadar" type=VLSD', 'channel 2': 'name="RailBend" type=VLSD', 'channel 3': 'name="ForceLevel" type=VLSD', 'channel 4': 'name="ProbeVolt" type=VALUE', 'channel 5': 'name="PadWear" type=VLSD', 'channel 6': 'name="FuelGauge" type=VALUE', 'channel 7': 'name="SpeedScan" type=VALUE', 'channel 8': 'name="TrafficScan" type=VALUE', 'channel 9': 'name="SensorForce" type=VLSD', 'channel 10': 'name="SnowModeSwitch" type=VLSD', 'channel 11': 'name="PMFiltration" type=VLSD', 'channel 12'

In [33]:
display(df)

,ProbeVolt,FuelGauge,SpeedScan,TrafficScan,Satellite,ASCDSet,Strut,StereoDepth,Drowsiness,DrivelineTorque,...,ExhaustFlow,HeadlightControl,OilLevel,Clutch,ValvePress,EmergencyBrakeAssist,RoofImpact,Pedal,BagTimer,DiffSpeed
timestamps,,,,,,,,,,,,,,,,,,,,,
2025-03-28 15:07:03.319909+01:00,4,61,52,50.088653,0.125537,184,421,8,1,220.889198,...,b'404.03255865711947',b'0.09418588924570548',b'52.06629822742478',b'99.10637402161633',b'5.80717256147574',b'OF',b'0.20337844532343385',b'50.937901680603794',b'8.480613122656841',b'568.9476529009735'
2025-03-28 15:07:04.319909+01:00,4,61,52,41.109035,0.879834,184,421,8,1,423.044214,...,b'221.3955786470791',b'0.510413337038713',b'89.26906933019788',b'43.1793642393552',b'3.8484673410478543',b'OF',b'0.2209055933912245',b'43.363827253557766',b'4.9840499904517985',b'530.2687216185486'
2025-03-28 15:07:05.319909+01:00,3,59,207,74.950983,0.803773,64,453,1,1,306.474417,...,b'95.83394779153286',b'0.7240610839757428',b'49.155815473467655',b'90.86941150125863',b'8.715160893868108',b'OF',b'0.643551411672695',b'6.502818117536535',b'3.850668484349976',b'522.9692164868554'
2025-03-28 15:07:06.319909+01:00,0,73,169,145.145889,0.346912,50,422,7,0,228.651875,...,b'192.68238796053782',b'0.5006173204369809',b'2.2359666185566174',b'30.47199430022305',b'3.953675479420408',b'ON',b'0.37990037066685534',b'31.462633725415046',b'1.7819296780854788',b'2528.0835257158014'
2025-03-28 15:07:07.319909+01:00,4,58,62,160.609544,0.440686,171,355,10,0,338.791536,...,b'139.11883508652002',b'0.4065584532436298',b'77.84110423902703',b'8.060507769815894',b'0.5797200786675905',b'OF',b'0.46136468102668193',b'88.01729549793625',b'4.715298399130053',b'2501.0144952081478'
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2025-03-28 15:16:59.319909+01:00,2,18,17,98.572129,0.025338,49,339,5,0,435.154203,...,b'4.9034133203600945',b'0.13932692214769293',b'17.391380404641097',b'47.32729338600804',b'6.877068705771721',b'OF',b'0.4196855299527269',b'27.971626004012496',b'2.1967801576732637',b'4202.966458696812'
2025-03-28 15:17:00.319909+01:00,5,29,161,2.551859,0.091563,154,105,2,0,294.173216,...,b'204.00687931109357',b'0.4658238808151748',b'93.37008618776078',b'50.52115258143214',b'9.414887584939674',b'ON',b'0.695178881020222',b'55.84787646721276',b'4.874620950415024',b'3544.2464273984842'
2025-03-28 15:17:01.319909+01:00,1,3,122,64.369780,0.602034,13,49,1,0,221.541297,...,b'413.82894607193685',b'0.505418592005912',b'42.741331499058134',b'60.5317163119725',b'7.975864714188995',b'OF',b'0.7513452968625951',b'4.6394725737524345',b'0.04406836156599869',b'3158.7598617889003'
